<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250721_14_40_%EC%86%90%EC%9D%B5%EA%B3%84%EC%82%B0%EC%84%9C(23%2C24%2C25)_%EC%B5%9C%EC%9D%80%ED%98%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:

!pip install dart-fss
!pip install OpenDartReader
!pip install FinanceDataReader
!pip install rapidfuzz
!pip install dart-fss OpenDartReader FinanceDataReader rapidfuzz


ERROR: Could not find a version that satisfies the requirement FinanceDataReader (from versions: none)
ERROR: No matching distribution found for FinanceDataReader
ERROR: Could not find a version that satisfies the requirement FinanceDataReader (from versions: none)
ERROR: No matching distribution found for FinanceDataReader


In [21]:

import os
import pandas as pd
from collections import defaultdict
from difflib import SequenceMatcher

# ✅ Step 1: 설정
import os
import pandas as pd
input_path = '/content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_2023, 2024, 2025'
output_path = '/content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_결과'
similarity_threshold = 0.6


In [22]:

# ✅ Step 2: 모든 파일에서 컬럼 수집
all_columns = []
file_columns = {}
for file in os.listdir(folder_path):
    if file.endswith('.xlsx'):
        try:
            df = pd.read_excel(os.path.join(folder_path, file), nrows=5)
            cols = list(df.columns)
            all_columns.extend(cols)
            file_columns[file] = cols
        except Exception as e:
            print(f"❌ {file}: {e}")



In [23]:
os.listdir(input_path)

['2023_사업보고서_03_포괄손익계산서_은행_연결_20241101.txt',
 '2023_사업보고서_02_손익계산서_20241101.txt',
 '2023_사업보고서_02_손익계산서_연결_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_증권_연결_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_연결_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_금융기타_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_은행_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_보험_연결_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_증권_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_금융기타_연결_20241101.txt',
 '2023_사업보고서_03_포괄손익계산서_보험_20241101.txt',
 '2024_사업보고서_03_포괄손익계산서_은행_연결_20250606.txt',
 '2024_사업보고서_03_포괄손익계산서_증권_20250606.txt',
 '2024_사업보고서_02_손익계산서_20250606.txt',
 '2024_사업보고서_03_포괄손익계산서_증권_여

In [26]:
import os

def extract_balance_sheet_indicators(input_path):
    indicators = set()
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            if '\t' in line:
                columns = line.strip().split('\t')
                for col in columns:
                    # 숫자가 아닌 컬럼명만 추출
                    if not any(char.isdigit() for char in col) and len(col.strip()) > 1:
                        indicators.add(col.strip())
    return indicators

In [41]:
import os

# 경로 및 대상 파일
input_path = '/content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_2023, 2024, 2025'
output_path = '/content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_결과'
target_files = [f for f in os.listdir(input_path) if "연결" in f and f.endswith(".txt")]

# 확인용: 첫 번째 파일만 미리 보기
if target_files:
    test_file = os.path.join(input_path, target_files[0])
    print(f"\n✅ 테스트 파일: {target_files[0]}\n")
    try:
        with open(test_file, encoding='utf-8') as f:
            for i in range(20):
                print(f"{i+1:02d}: {f.readline().strip()}")
    except UnicodeDecodeError:
        with open(test_file, encoding='cp949') as f:
            for i in range(20):
                print(f"{i+1:02d}: {f.readline().strip()}")
else:
    print("❌ '연결' 포함된 txt 파일이 없습니다.")


❌ '연결' 포함된 txt 파일이 없습니다.


In [42]:
import unicodedata
import os

# 경로 설정
input_path

# 파일명 유니코드 정규화 후 '연결' 포함 여부 판단
all_files = os.listdir(input_path)
target_files = [
    f for f in all_files if "연결" in unicodedata.normalize('NFC', f) and f.endswith('.txt')
]

# 결과 확인
print("📂 '연결' 포함된 txt 파일 개수:", len(target_files))
for f in target_files:
    print(" -", f)


📂 '연결' 포함된 txt 파일 개수: 18
 - 2023_사업보고서_03_포괄손익계산서_은행_연결_20241101.txt
 - 2023_사업보고서_02_손익계산서_연결_20241101.txt
 - 2023_사업보고서_03_포괄손익계산서_증권_연결_20241101.txt
 - 2023_사업보고서_03_포괄손익계산서_연결_20241101.txt
 - 2023_사업보고서_03_포괄손익계산서_보험_연결_20241101.txt
 - 2023_사업보고서_03_포괄손익계산서_금융기타_연결_20241101.txt
 - 2024_사업보고서_03_포괄손익계산서_은행_연결_20250606.txt
 - 2024_사업보고서_03_포괄손익계산서_증권_연결_20250606.txt
 - 2024_사업보고서_03_포괄손익계산서_금융기타_연결_20250606.txt
 - 2024_사업보고서_03_포괄손익계산서_보험_연결_20250606.txt
 - 2024_사업보고서_03_포괄손익계산서_연결_20250606.txt
 - 2024_사업보고서_02_손익계산서_연결_20250606.txt
 - 2025_1분기보고서_02_손익계산서_연결_20250606.txt
 - 2025_1분기보고서_03_포괄손익계산서_금융기타_연결_20250606.txt
 - 2025_1분기보고서_03_포괄손익계산서_은ᄒ

In [43]:
# 첫 번째 파일 경로 설정
import unicodedata
test_file = os.path.join(input_path, target_files[0])

# 20줄만 출력해서 포맷 확인
try:
    with open(test_file, encoding='utf-8') as f:
        print(f"\n🧾 파일: {unicodedata.normalize('NFC', target_files[0])}\n")
        for i in range(20):
            print(f"{i+1:02d}: {f.readline().strip()}")
except UnicodeDecodeError:
    with open(test_file, encoding='cp949') as f:
        print(f"\n🧾 파일 (cp949): {unicodedata.normalize('NFC', target_files[0])}\n")
        for i in range(20):
            print(f"{i+1:02d}: {f.readline().strip()}")



🧾 파일: 2023_사업보고서_03_포괄손익계산서_은행_연결_20241101.txt


🧾 파일 (cp949): 2023_사업보고서_03_포괄손익계산서_은행_연결_20241101.txt

01: 재무제표종류	종목코드	회사명	시장구분	업종	업종명	결산월	결산기준일	보고서종류	통화	항목코드	항목명	당기	전기	전전기
02: 포괄손익계산서, 은행 - 연결	[138930]	BNK금융지주	유가증권시장상장법인	649	기타 금융업	12	2023-12-31	사업보고서	KRW	dart_GainLossFromFinancialInstrumentsAtAmortisedCost	      상각후원가측정금융상품관련손익	-245,957,046	283,596,929	2,442,858,016
03: 포괄손익계산서, 은행 - 연결	[138930]	BNK금융지주	유가증권시장상장법인	649	기타 금융업	12	2023-12-31	사업보고서	KRW	dart_GainLossFromFinancialInstrumentsAtFairValueThroughOtherComprehensiveIncome	      기타포괄손익-공정가치측정금융상품관련손익	4,798,390,870	2,808,705,725	12,719,144,999
04: 포괄손익계산서, 은행 - 연결	[138930]	BNK금융지주	유가증권시장상장법인	649	기타 금융업	12	2023-12-31	사업보고서	KRW	dart_GainLossFromFinancialInstrumentsAtFairValueThroughProfitOrLoss	      당기손익-공정가치측정금융상품관련손익	161,584,183,296	-4,287,240,484	176,281,875,491
05: 포괄손익계산서, 은행 - 연결	[138930]	BNK금융지주	유가증권시장상장법인	649	기타 금융업	12	2023-12-31	사업보고서	KRW	dart_NonOperatingProfitLoss	      영업외손익	74,386,087,514	-30,938,525,501	10,018,134,

In [30]:
!pip install rapidfuzz tqdm

In [44]:
import os
import unicodedata
from collections import defaultdict, Counter
from tqdm import tqdm
from rapidfuzz import process, fuzz

# ✅ 경로 설정
input_path
output_path
# ✅ 항목명 추출 함수 (12번째 필드: index 11이 항목명)
def extract_column_names(filepath):
    column_names = []
    try:
        with open(filepath, encoding='utf-8') as f:
            for i, line in enumerate(f):
                parts = line.strip().split('\t')
                if len(parts) > 11 and i > 0:
                    name = parts[11].strip()
                    if name and name != '항목명':
                        column_names.append(name)
    except UnicodeDecodeError:
        with open(filepath, encoding='cp949') as f:
            for i, line in enumerate(f):
                parts = line.strip().split('\t')
                if len(parts) > 11 and i > 0:
                    name = parts[11].strip()
                    if name and name != '항목명':
                        column_names.append(name)
    return column_names

# ✅ 연결 포함된 .txt 파일 추출
all_files = os.listdir(input_path)
target_files = [
    f for f in all_files if "연결" in unicodedata.normalize('NFC', f) and f.endswith(".txt")
]
print(f"📁 '연결' 포함된 파일 수: {len(target_files)}개")

# ✅ 항목명 수집
all_column_names = []
for idx, file in enumerate(target_files, 1):
    file_path = os.path.join(input_path, file)
    print(f"  [{idx}/{len(target_files)}] → {file} 처리 중")
    all_column_names.extend(extract_column_names(file_path))

print(f"✅ 총 추출된 항목명 수: {len(all_column_names)}개")

# ✅ 중복 제거
unique_columns = list(set(all_column_names))
print(f"🧹 중복 제거 후 유니크 항목명 수: {len(unique_columns)}개")

# ✅ 유사 컬럼 그룹화
grouped = defaultdict(list)
visited = set()

print("🧠 유사 컬럼 그룹화 진행 중...")
for col in tqdm(unique_columns):
    if col in visited:
        continue
    matches = process.extract(
        col, unique_columns,
        scorer=fuzz.token_sort_ratio,
        score_cutoff=75,
        limit=20
    )
    matched_names = [m[0] for m in matches]
    for m in matched_names:
        visited.add(m)
    grouped[col].extend(matched_names)

# ✅ 대표 컬럼 추출
freq = Counter(all_column_names)
representatives = [max(members, key=lambda x: freq[x]) for members in grouped.values()]
print(f"🏷 대표컬럼 추출 완료: {len(set(representatives))}개")

# ✅ 결과 저장
import pandas as pd
os.makedirs(output_path, exist_ok=True)
df = pd.DataFrame({'대표컬럼': sorted(set(representatives))})
df.to_csv(os.path.join(output_path, "대표컬럼목록.csv"), index=False)
print("💾 대표컬럼 CSV 저장 완료")


📁 '연결' 포함된 파일 수: 18개
  [1/18] → 2023_사업보고서_03_포괄손익계산서_은행_연결_20241101.txt 처리 중
  [2/18] → 2023_사업보고서_02_손익계산서_연결_20241101.txt 처리 중
  [3/18] → 2023_사업보고서_03_포괄손익계산서_증권_연결_20241101.txt 처리 중
  [4/18] → 2023_사업보고서_03_포괄손익계산서_연결_20241101.txt 처리 중
  [5/18] → 2023_사업보고서_03_포괄손익계산서_보험_연결_20241101.txt 처리 중
  [6/18] → 2023_사업보고서_03_포괄손익계산서_금융기타_연결_20241101.txt 처리 중
  [7/18] → 2024_사업보고서_03_포괄손익계산서_은행_연결_20250606.txt 처리 중
  [8/18] → 2024_사업보고서_03_포괄손익계산서_증권_연결_20250606.txt 처리 중
  [9/18] → 2024_사업보고서_03_포괄손익계산서_금융기타_연결_20250606.txt 처리 중
  [10/18] → 2024_사업보고서_03_포괄손익계산서_보험_연결_20250606.txt 처리 중
  [11/18] → 2024_사업보고서_03_포괄손익계산서_연결_20250606.txt 처리 중
  [12/18] → 2024_사업보고서_02_손익계산서_연결_20250606.txt 처리 중
  [13/18] → 2025_1분기ᄇ

100%|██████████| 6069/6069 [00:07<00:00, 770.59it/s] 


🏷 대표컬럼 추출 완료: 1269개
💾 대표컬럼 CSV 저장 완료


In [45]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_결과/대표컬럼목록.csv")
print(df.columns)


Index(['대표컬럼'], dtype='object')


In [46]:
import pandas as pd
from collections import defaultdict, Counter
from rapidfuzz import process, fuzz
from tqdm import tqdm
import os

# ✅ 경로 설정
output_path
column_file_path = os.path.join(output_path, "대표컬럼목록.csv")  # 여기에 원본 컬럼 리스트가 있다고 가정
output_csv_path = os.path.join(output_path, "대표컬럼_최종정리.csv")

# ✅ 1. 컬럼 리스트 불러오기
def load_column_list(path, col_name= "대표컬럼"):
    df = pd.read_csv(path)
    return df[col_name].dropna().tolist()

# ✅ 2. 유사 컬럼끼리 그룹 묶기 (RapidFuzz 기반)
def group_similar_columns(column_list, similarity_cutoff=75):
    unique_columns = list(set(column_list))
    grouped = defaultdict(list)
    visited = set()

    for col in tqdm(unique_columns, desc="🧠 유사 컬럼 그룹화 진행 중"):
        if col in visited:
            continue
        matches = process.extract(
            col, unique_columns,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=similarity_cutoff,
            limit=20
        )
        matched_names = [m[0] for m in matches]
        for m in matched_names:
            visited.add(m)
        grouped[col].extend(matched_names)

    return grouped

# ✅ 3. 각 그룹에서 대표 컬럼 선택 (빈도 기준)
def select_representative_columns(grouped_dict, full_column_list):
    frequency = Counter(full_column_list)
    representative_mapping = dict()

    for rep, members in grouped_dict.items():
        best = max(members, key=lambda x: frequency[x])
        for m in members:
            representative_mapping[m] = best  # 유사컬럼 → 대표컬럼

    return representative_mapping

# ✅ 4. 실행
column_list = load_column_list(column_file_path)
grouped_columns = group_similar_columns(column_list, similarity_cutoff=75)
representative_map = select_representative_columns(grouped_columns, column_list)

# ✅ 5. 결과 저장
# 대표컬럼 리스트만 정리
final_representatives = sorted(set(representative_map.values()))
rep_df = pd.DataFrame({"대표컬럼": final_representatives})
rep_df.to_csv(output_csv_path, index=False)

print(f"✅ 대표컬럼 {len(rep_df)}개가 추출되어 저장되었습니다.")


🧠 유사 컬럼 그룹화 진행 중: 100%|██████████| 1269/1269 [00:00<00:00, 2046.56it/s]

✅ 대표컬럼 954개가 추출되어 저장되었습니다.


In [47]:
# ✅ 6. 전체 컬럼 → 대표컬럼으로 치환
replaced_columns = [representative_map.get(col, col) for col in column_list]  # 매핑 안 된 경우 본인 유지

# ✅ 7. 대표컬럼별 사용 빈도 집계 및 정렬
representative_counter = Counter(replaced_columns)
sorted_representatives = representative_counter.most_common()

# ✅ 8. 정렬된 대표컬럼 + 빈도수 저장
freq_df = pd.DataFrame(sorted_representatives, columns=["대표컬럼", "빈도수"])
freq_output_path = os.path.join(output_path, "대표컬럼_빈도순정렬.csv")
freq_df.to_csv(freq_output_path, index=False)

# ✅ 9. 출력 확인
print("📊 대표컬럼별 사용 빈도 (내림차순 정렬):")
for col, freq in sorted_representatives[:20]:  # 상위 20개만 보기 좋게 출력
    print(f"{col}: {freq}")

print(f"\n✅ 빈도 기준 정렬 결과가 {freq_output_path}에 저장되었습니다.")


📊 대표컬럼별 사용 빈도 (내림차순 정렬):
기타포괄손익-공정가치지분상품평가손익: 13
기타포괄손익-공정가치 측정 금융자산 평가손익: 7
비지배지분 귀속 당기순이익: 7
계속영업기본주당이익: 6
공정가치측정금융자산평가손익: 6
당기총포괄이익(손실): 6
지배주주지분순이익: 6
기본주당이익(손실): 5
당기손익-공정가치측정금융상품관련손실: 5
후속적으로 당기손익으로 재분류되지 않는 항목(세후기타포괄손익): 5
법인세비용차감전순이익: 5
지배기업의 소유주에게 귀속되는 당기순이익: 5
계속영업 희석주당순손익: 4
중단영업당기순이익: 4
관계기업관련이익: 4
관계기업 기타포괄손익지분: 4
관계기업 기타포괄손익에 대한 평가이익: 4
확정급여채무의 재측정요소: 4
관계기업투자이익(손실): 4
기타영업비용: 4

✅ 빈도 기준 정렬 결과가 /content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_결과/대표컬럼_빈도순정렬.csv에 저장되었습니다.


In [48]:
# ✅ 10. 대표컬럼 → 유사컬럼 리스트 역변환
representative_to_similars = defaultdict(list)
for similar_col, rep_col in representative_map.items():
    representative_to_similars[rep_col].append(similar_col)

# ✅ 11. 딕셔너리를 CSV로 저장
rep_group_path = os.path.join(output_path, "대표컬럼_유사컬럼_매핑.csv")
rep_group_df = pd.DataFrame([
    {"대표컬럼": rep, "유사컬럼들": ", ".join(sorted(list(set(similars))))}
    for rep, similars in representative_to_similars.items()
])
rep_group_df.to_csv(rep_group_path, index=False)

print(f"✅ 대표컬럼별 유사컬럼 목록이 {rep_group_path}에 저장되었습니다.")


✅ 대표컬럼별 유사컬럼 목록이 /content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_결과/대표컬럼_유사컬럼_매핑.csv에 저장되었습니다.


In [49]:
# ✅ 12. 대표컬럼별 전체 사용 빈도 계산 (유사컬럼 등장 횟수 합산 기준)
column_frequency = Counter(column_list)  # 원본 컬럼 리스트에서의 빈도

rep_count_list = []
for rep, similars in representative_to_similars.items():
    total_count = sum(column_frequency[col] for col in similars if col in column_frequency)
    rep_count_list.append((rep, total_count, similars))

# ✅ 내림차순 정렬
rep_count_list_sorted = sorted(rep_count_list, key=lambda x: x[1], reverse=True)

# ✅ 13. 정리된 형태로 DataFrame 저장
rep_count_df = pd.DataFrame([
    {
        "대표컬럼": rep,
        "총사용빈도": count,
        "유사컬럼수": len(similars),
        "유사컬럼목록": ", ".join(sorted(set(similars)))
    }
    for rep, count, similars in rep_count_list_sorted
])

rep_count_path = os.path.join(output_path, "대표컬럼_사용빈도_내림차순.csv")
rep_count_df.to_csv(rep_count_path, index=False)

print(f"✅ 대표컬럼 사용빈도 기준 정리 결과가 {rep_count_path}에 저장되었습니다.")


✅ 대표컬럼 사용빈도 기준 정리 결과가 /content/drive/MyDrive/Colab Notebooks/실전프로젝트/손익계산서_2023, 2024, 2025/손익계산서_결과/대표컬럼_사용빈도_내림차순.csv에 저장되었습니다.
